In [154]:
from datasets import load_dataset

datasetraw = load_dataset("climate_fever")

In [155]:
dataa = datasetraw.map(remove_columns=["claim_id","evidences"])

dataset = dataa["test"].train_test_split(test_size=0.2)

In [156]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")


def tokenize_function(examples):
    return tokenizer(examples["claim"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

c:\Users\enzol\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 307/307 [00:00<00:00, 4077.09 examples/s]


In [157]:
traindataset = tokenized_datasets["train"]
evaldataset = tokenized_datasets['test']

print(traindataset)

Dataset({
    features: ['claim', 'claim_label', 'input_ids', 'attention_mask'],
    num_rows: 1228
})


In [158]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=4)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [159]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [160]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [161]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer")

In [162]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=traindataset,
    eval_dataset=evaldataset,
    compute_metrics=compute_metrics,
)

  0%|          | 0/462 [37:04<?, ?it/s]


In [163]:
print(traindataset)
print(evaldataset)

Dataset({
    features: ['claim', 'claim_label', 'input_ids', 'attention_mask'],
    num_rows: 1228
})
Dataset({
    features: ['claim', 'claim_label', 'input_ids', 'attention_mask'],
    num_rows: 307
})


In [164]:
trainer.train()

  0%|          | 0/462 [00:00<?, ?it/s]

ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.